# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
!pip install textdistance

In [2]:
import os, re
from string import punctuation
import numpy as np
from collections import Counter
punctuation += "«»—…“”"
punct = set(punctuation)
from string import punctuation
import numpy as np
from collections import Counter
from tqdm import tqdm
import textdistance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [3]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [5]:
vocab = Counter(re.findall('\w+', corpus.lower()))

In [6]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [7]:
X.shape

(368802, 10000)

In [8]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [9]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [10]:
# ваш код тут
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def my_get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    for i in range(len(closest)):
        if closest[i][1] == closest[i+1][1]:
            if P(closest[i][0]) > P(closest[i+1][0]):
                closest.pop(i+1)
                return closest
            else:
                closest.pop(i)
                return closest
        else:
            return closest
    
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


Результат работы обычной функции:

In [11]:
get_closest_hybrid_match("людис", X, vec)

[('люди', 0.8), ('лдис', 0.8), ('людини', 0.6666666666666667)]

Результат работы функции с выбором наиболее вероятного кандидата, если расстояние исправления одинаковое:

In [12]:
my_get_closest_hybrid_match("людис", X, vec)

[('люди', 0.8), ('людини', 0.6666666666666667)]

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [13]:
# словарь правильных слов

vocab = Counter(re.findall('\w+', corpus.lower()))

In [14]:
# словарь удалений

def dels(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    return set(deletes)

dels_dict = {}

for word in vocab:
    for d in dels(word):
        dels_dict[d] = word

In [15]:
len(dels_dict)

2872459

In [16]:
# подбор вероятного слова

def correction(word): 
    return max(candidates(word), key=P)

def candidates(word): 
    return (known([word]) or known(dels(word)) or [word])

# сравнение слова со словарем удалений

def known(words): 
    return set(w for w in words if w in dels_dict)

In [17]:
correction("дререво")

'дерево'

In [18]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

Оценка тремя метриками:

In [19]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

100%|█████████████████████████████████████████████████████████████████████████████| 915/915 [00:00<00:00, 12336.62it/s]


In [20]:
print(f"Процент правильных слов: {correct/total}")
print(f"Процент исправленных ошибок: {mistaken_fixed/total_mistaken}")
print(f"Процент ошибочно исправленных правильных слов: {correct_broken/total_correct}")

Процент правильных слов: 0.607503751875938
Процент исправленных ошибок: 0.055900621118012424
Процент ошибочно исправленных правильных слов: 0.31089927644424026
